In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import threading
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from queue import Queue

# Croma

In [2]:
class Croma:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.page_visit =
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = self.s)
        self.driver.get('https://www.croma.com/')
        self.flag = False

        while self.flag != True:
            try:
                #self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
                self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="searchV2"]')))

                self.un.send_keys(self.search)
                self.flag = True
            except (ElementNotInteractableException, TimeoutException) :
                self.driver.refresh()
                time.sleep(3)
                pass
#                 self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#                 self.un.send_keys(self.search)
#                 self.flag = True
    

    
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)
#         try:
#             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#             #time.sleep(2)
#             self.un.send_keys(search)
#             time.sleep(2)
#             self.un.send_keys(Keys.ENTER)
#         except ElementNotInteractableException:
#             self.driver.refresh()
#             self.un = WebDriverWait(self.driver, 12).until(EC.presence_of_element_located((By.CLASS_NAME,'MuiAutocomplete-input.MuiAutocomplete-inputFocused.search-field')))
#             #time.sleep(2)
#             self.un.send_keys(search)
#             time.sleep(2)
#             self.un.send_keys(Keys.ENTER)
            
        
    def link_collection(self):
    
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        review_link = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.PARTIAL_LINK_TEXT, self.search)))
        #name = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))


        d = {}
        name = []
        for i in review_link:
            name.append(i.text)

        for i, j in  zip(review_link, name):
            x = self.search.lower()
            if x in j.lower() and '(Renewed)' not in j:
                d[i] = j

        time.sleep(1)
        return d
    
    def scrapings(self, d, limit):
        j = 1
        prod_review= {}
        link = {}
        price = {}
        ratings = {}
        avg_rating = {}
        
        for i in d:
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            #i.click()
            webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()
            while len(self.driver.window_handles) != 2:
                time.sleep(3)
                webdriver.ActionChains(self.driver).move_to_element(i).click(i).perform()

            #driver.execute_script("arguments[0].click();", i)
            time.sleep(2)
            #driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)
            
            y = d[i]
            # scraping link
            l = self.driver.current_url
            link[y] = l
            
            
            # scraping price
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')
            p = []
            h = x.find_all('span', {'class' : 'amount'})
            
            if len(h) <= 0:
                price[y] = p
            else:
                p.append(h[0].text)
                price[y] = p
            
            
            #time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            #try:
            #    all_comments = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,'btn-wrap.view-all-review-btn')))
            #all_comments[0].click()
            #   webdriver.ActionChains(driver).move_to_element(all_comments).click(all_comments).perform
            #except TimeoutException:
            #    pass
            
            # scrapping ratings
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            r = []
            rating = {'5 Stars': 0, '4 Stars' : 0, '3 Stars': 0, '2 Stars': 0, '1 Stars': 0}
            l = h = x.find_all('div', {'class' : 'barAndStar'})
            # h = x.find_all('p', {'class' : 'right-text'})
            # rating['4 Stars'] = l[0].text
            if len(l) <= 0:
                ratings[y] = rating
            else:
                count = 5
                for i in l:
                    rating['{} Stars'.format(count)] = i.text[6:]
                    count-=1
                    
                ratings[y] = rating
            
            
            # scraping  avg ratings
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            t = []
            h = x.find_all('div', {'class' : 'totalRating'})
            
            if len(h) <= 0:
                avg_rating[y] = t
            else:
                t.append(h[0].text)
                avg_rating[y] = t

            #time.sleep(3)
            
            
            # scraping review
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class' : 'desc'})
            reviews = set()
            #y = d[i]
            for k in h:
                reviews.add(k.text)
                print(k.text)
                if len(reviews) == self.limit:
                    break
            #driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            #curr_height = driver.execute_script('return document.body.scrollHeight;')
            flag = 0
            reviews1 = reviews
            while len(reviews) != self.limit:
                try:
                    #driver.execute_script('window.scrollBy(0, 2000);')
                    #time.sleep(2)
                    flag+=1
                    more = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, 'View All Reviews')))
                    webdriver.ActionChains(self.driver).move_to_element(more).click(more).perform()
                    e = self.driver.page_source
                    x= BeautifulSoup(e, 'html.parser')
                    t = x.find_all('div', {'class' : 'desc'})
                    #print(curr_height)
                    #curr_height += 100
                    for l in t:
                        reviews.add(l.text.strip())
                        if len(reviews) == self.limit:
                            break

                    self.driver.execute_script('window.scrollBy(5350, 100);')
                    time.sleep(3)
                    print(flag)
                    if flag == 11:
                        if len(reviews1) == len(reviews):
                            break

                    #driver.execute_script('window.scrollBy(0, 2000);')
                except TimeoutException:
                    flag+=3
                    if flag >= 11 :
                        break
                    else:
                        pass

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])


            print(j)
            j+=1

        return prod_review, ratings, link, avg_rating, price
        
        
    def run(self):
        links = []
        reviews = []
        all_ratings = []
        prices = []
        avg_r = []
        
        main_d = {}
        now = time.time()
        for i in range(3):
            d = self.link_collection()
            print(len(d))
            new_d = {}
            for i, j in d.items():
                if i in main_d:
                    continue
                else:
                    new_d[i] = j
            for i, j in d.items():
                main_d[i] = j
            x, y, z, a, b = self.scrapings(new_d, self.limit)
            reviews.append(x)
            all_ratings.append(y)
            links.append(z)
            avg_r.append(a)
            prices.append(b)
            
            
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(3)
            next_page =  WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn.btn-secondary.btn-viewmore')))
            webdriver.ActionChains(self.driver).move_to_element(next_page).click(next_page).perform()
        
        self.result_queue.put((reviews, prices, avg_r, all_ratings, links))
        
        #return reviews, all_ratings, links, avg_r, prices
#x = Croma('iPhone 14', 10)

    
    

# Amazon

In [3]:
class Amazon:
    def __init__(self, search, limit, page_visit):
        self.search = search
        self.limit = limit
        self.result_queue = Queue()
        self.page_visit = page_visit
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=self.s)
        self.driver.get('https://www.amazon.in/')
        self.un = WebDriverWait(self.driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="twotabsearchtextbox"]')))
        self.un.send_keys(search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)

    def link_collection(self):
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        first_page_id = self.driver.current_window_handle
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal'))) # a-size-base.s-underline-text
        reviews = []
        price = []

        for i in review:
            if '₹' not in i.text:
                reviews.append(i)

        for i in review:
            if '₹' in i.text:
                price.append(i.text)
        #price = []

        cnt = 0
        for i in price:
            if cnt%2 != 0:
                price.append(i)
            cnt+=1

        try:
            title1 = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-size-base-plus.a-color-base.a-text-normal')))
        except TimeoutException:
            title1 = []

        try: 
            title2 = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'a-size-medium.a-color-base.a-text-normal')))
        except TimeoutException:
            title2 = []


        main_title = title1 if len(title1) > len(title2) else title2


        titles = [] 
        d = {}
        prices = {}
        for i, j in zip(main_title, reviews):
            x = self.search.lower()
            if ('(Renewed)' not in i.text) or ('WeConnect' not in i.text) or ('Protect+' not in i.text): # or ('AppleCare' not in i.text) 
                d[j] = i.text
                titles.append(i.text)

        for i, j in zip(main_title, price):
            prices[i.text] = j[1:]


        time.sleep(1)
        return d, titles, prices




    def scraping(self, d, limit, prices):
        # j = 1
        prod_review = {}
        ratings = {}
        link_of_product = {}
        overall_rating = {}
        for i in d:
            y = d[i]
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            # driver.switch_to.window(first_page_id)
            i.click()
            time.sleep(2)
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)

            link = self.driver.current_url
            link_of_product[y] = link

        

            time.sleep(6)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')

            try:
                all_comments = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'a-link-emphasis.a-text-bold')))
                all_comments.click()
                time.sleep(2)
            except TimeoutException:
                pass

            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            try:
                box = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, 'a-dropdown-prompt')))
                for k in box:
                    if k.text == 'All formats':
                        k.click()
            except TimeoutException:
                pass

            try:
                click = WebDriverWait(self.driver, 10).until(
                    EC.presence_of_element_located((By.ID, 'format-type-dropdown_1')))
                click.click()
                time.sleep(3)
            except TimeoutException:
                pass

             # scrapping views
            e = self.driver.page_source
            x = BeautifulSoup(e, 'html.parser')

            # scraping rating
            h = x.find_all('span', {'class': 'a-size-medium a-color-base'})

            clean = ' '
            for g in h:
                cleaned = g.text.strip()
                if g.text.strip() != '':
                    clean += cleaned[:3]
            overall_rating[y] = cleaned.strip()

            ## scraping review

            t = x.find_all('span', {'class': 'a-size-base review-text review-text-content'})
            reviews = set()
            for l in t:
                reviews.add(l.text.strip())
                if len(reviews) == self.limit:
                    break

            # star ratings
            h = x.find_all('td', {'class': 'a-text-right a-nowrap'})
            rating = {}
            counter = 5
            for rat in h:
                rating['{} Stars'.format(counter)] = rat.text.strip()
                counter -= 1

            ratings[y] = rating


            if len(reviews) != self.limit or len(reviews) < self.limit:
                try:
                    n = 0
                    while len(reviews) != self.limit:
                        try:
                            last = WebDriverWait(self.driver, 5).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-disabled.a-last')))
                            x = last.text
                            if x == 'Next page':
                                break
                        except (TimeoutException, StaleElementReferenceException):
                            review1 = reviews
                            next_ = WebDriverWait(self.driver, 10).until(
                                EC.presence_of_element_located((By.CLASS_NAME, 'a-last')))
                            next_.click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x = BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class': 'a-size-base review-text review-text-content'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == self.limit:
                                    break
                            prod_review[y] = reviews
                            if len(reviews) == len(review1):
                                break
                except TimeoutException:
                    pass
            else:
                prod_review[y] = reviews

            prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])

        return prod_review, ratings, link_of_product, overall_rating

    def run(self, page_visit):
        review_of_product = []
        ratings_of_product = []
        link_product = []
        price_product = []
        overall_r = []

        for i in range(self.page_visit):
            d, titles, prices = self.link_collection()
            print(titles)
            prod, ratings, links, overall = self.scraping(d, self.limit, prices)

            review_of_product.append(prod)
            ratings_of_product.append(ratings)
            link_product.append(links)
            price_product.append(prices)
            overall_r.append(overall)


            self.driver.switch_to.window(self.driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(2)
            flag = False
            while flag != True:
                try:
                    next_ = WebDriverWait(self.driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME,
                                                                                                's-pagination-item.s-pagination-next.s-pagination-button.s-pagination-separator')))
                    next_.click()
                    flag = True
                except (TimeoutException, ElementNotInteractableException, ElementClickInterceptedException):
                    self.driver.refresh()
                    time.sleep(3)
                    pass
                
        self.result_queue.put((review_of_product,  price_product, overall_r, ratings_of_product, link_product))
        #return review_of_product, ratings_of_product, link_product, price_product, overall_r



# Flipkart

In [4]:
class Flipkart:
    def __init__(self, search, limit):
        self.search = search
        self.limit = limit
        self.result_queue = Queue()
        self.s = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service = self.s)
        self.driver.get('https://www.flipkart.com/')
        
        x_button = WebDriverWait(self.driver, 17).until(EC.presence_of_element_located((By.CLASS_NAME, '_2KpZ6l._2doB4z'))) 
        x_button.click()
        self.un = WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="container"]/div/div[1]/div[1]/div[2]/div[2]/form/div/div/input')))
        self.un.send_keys(self.search)
        time.sleep(2)
        self.un.send_keys(Keys.ENTER)
        
    
    def link_collection(self):
    
        self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
        time.sleep(2)
        review = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_1fQZEK')))
        title = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME,'_4rR01T')))

        titles = [] 
        d = {}
        reviews = []

        try:
            for i in review:
                if '(' in i.text:
                    reviews.append(i)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()
        try:
            for i, j in zip(title, reviews):
                #x = search.lower()
                if '(Renewed)' not in i.text:
                    d[j] = i.text
                    titles.append(i.text)
        except StaleElementReferenceException:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()

        if len(d) == 0:
            self.driver.refresh()
            time.sleep(2)
            self.link_collection()



        time.sleep(1)
        return d
    
    
    def scraping(self, d, limit):
        j = 1
        prod_review= {}
        link = {}
        overall_rating = {}
        price = {}
        ratings = {}
        for i in d:
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            self.driver.switch_to.window(self.driver.window_handles[0])
            i.click()
            time.sleep(2)
            #driver.switch_to.window(driver.window_handles[j])
            c = self.driver.window_handles[1]
            self.driver.switch_to.window(c)
            
            
            y = d[i]
        
            # scraping link
            url = self.driver.current_url
            link[y] = url
            
            
            # scraping stars
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class' : '_1uJVNT'})
            stars = {}
            count = 5
            for i in h:
                stars["{} Stars".format(count)] = i.text
                count-=1

            ratings[y] = stars


            #scraping price   <div class="_30jeq3 _16Jk6d">₹69,999</div>
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class' : '_30jeq3 _16Jk6d'})
            p = []
            for i in h:
                p.append(i.text)
            price[y] = p


            # average ratings
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')
            h = x.find_all('div', {'class' : '_2d4LTz'})
            r = []
            for i in h:
                r.append(i.text)
            overall_rating[y] = r
            
            
            #time.sleep(2)
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            time.sleep(7)
            all_comments_link = WebDriverWait(self.driver, 6).until(EC.presence_of_element_located((By.CLASS_NAME, '_3UAT2v._16PBlm'))) 
            self.driver.execute_script("arguments[0].click();", all_comments_link)

            #time.sleep(3)

            # scraping review
            e = self.driver.page_source
            x= BeautifulSoup(e, 'html.parser')

            h = x.find_all('div', {'class' : 't-ZTKy'})
            reviews = set()
           # y = d[i]
            for k in h:
                reviews.add(k.text)
                if len(reviews) == limit:
                    break


            if len(reviews) != self.limit or len(reviews) < self.limit:
                    try:
                        while len(reviews) != self.limit:
                            review1 = reviews
                            next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                            if len(next_page) == 1:
                                next_page[0].click()
                            elif len(next_page) == 2:
                                next_page[1].click()
                            time.sleep(4)
                            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                            e = self.driver.page_source
                            x= BeautifulSoup(e, 'html.parser')
                            t = x.find_all('div', {'class' : 't-ZTKy'})
                            for l in t:
                                reviews.add(l.text.strip())
                                if len(reviews) == limit:
                                    break
                            prod_review[y] = reviews

                    except TimeoutException:
                        pass
            else:
                prod_review[y] = reviews

            #prod_review[y] = reviews
            self.driver.close()
            self.driver.switch_to.window(self.driver.window_handles[0])


            print(j)
            j+=1

        return prod_review, link, overall_rating, price, ratings

    
    def run(self):
        review = []
        avg_rating = []
        prices = []
        links = []
        all_r = []
        for i in range(3):
            d = self.link_collection()
            print(len(d))
            x, y, z , a, b = self.scraping(d, self.limit)
            review.append(x)
            links.append(y)
            avg_rating.append(z)
            prices.append(a)
            all_r.append(b)
            
            #driver.switch_to.window(driver.window_handles[0])
            self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
            next_page = WebDriverWait(self.driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
            try:
                if len(next_page) == 1:
                    next_page[0].click()
                elif len(next_page) == 2:
                    next_page[1].click()
            except ElementClickInterceptedException:
                self.driver.refresh()
                time.sleep(4)
                self.driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
                
                flag = False
                while flag != True:
                    try:
                        next_page = WebDriverWait(self.driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, '_1LKTO3')))
                        if len(next_page) == 1:
                            next_page[0].click()
                        elif len(next_page) == 2:
                            next_page[1].click()
                        flag = True
                    except (ElementNotInteractableException, TimeoutException, ElementClickInterceptedException):
                        self.driver.refresh()
                        time.sleep(3)
                        pass
                    
        self.result_queue.put((review, prices, avg_rating, all_r, links))

        #return review, avg_rating, prices, links, all_r

In [6]:
x = Croma('iPhone 14', 10,1)
y = Flipkart('iPhone 14', 10,1)
z = Amazon('iPhone 14', 10, 1)

t1 = threading.Thread(target=x.run(1))
t2 = threading.Thread(target=y.run(1))
t3 = threading.Thread(target=z.run(1))

t1.start()
t2.start()
t3.start()

t1.join()
t2.join()
t3.join()


TypeError: __init__() takes 3 positional arguments but 4 were given

In [7]:
review = []
price = []
avg_rating = []
all_rating = []
link = []

for i in [x, y, z]:
    x, y, z, a, b = i.result_queue.get()
    review.append(x)
    price.append(y)
    avg_rating.append(z)
    all_rating.append(a)
    link.append(b)

In [9]:
all_rating

[[{'Apple iPhone 14 (128GB, Blue)': {'5 Stars': '172',
    '4 Stars': '20',
    '3 Stars': '2',
    '2 Stars': '4',
    '1 Stars': '6'},
   'Apple iPhone 14 (128GB, Starlight)': {'5 Stars': '87',
    '4 Stars': '9',
    '3 Stars': '1',
    '2 Stars': '2',
    '1 Stars': '2'},
   'Apple iPhone 14 (128GB, Purple)': {'5 Stars': '98',
    '4 Stars': '7',
    '3 Stars': '2',
    '2 Stars': '1',
    '1 Stars': 0},
   'Apple iPhone 14 (128GB, Midnight)': {'5 Stars': '106',
    '4 Stars': '18',
    '3 Stars': '2',
    '2 Stars': '3',
    '1 Stars': 0},
   'Apple iPhone 14 (256GB, Blue)': {'5 Stars': '28',
    '4 Stars': '1',
    '3 Stars': '1',
    '2 Stars': 0,
    '1 Stars': 0},
   'Apple iPhone 14 (128GB, Red)': {'5 Stars': '24',
    '4 Stars': '2',
    '3 Stars': '1',
    '2 Stars': 0,
    '1 Stars': 0},
   'Apple iPhone 14 (256GB, Red)': {'5 Stars': '2',
    '4 Stars': '2',
    '3 Stars': 0,
    '2 Stars': 0,
    '1 Stars': 0},
   'Apple iPhone 14 (256GB, Purple)': {'5 Stars': '12',
    '

In [10]:
all_rating

[[{'Apple iPhone 14 (128GB, Blue)': {'5 Stars': '172',
    '4 Stars': '20',
    '3 Stars': '2',
    '2 Stars': '4',
    '1 Stars': '6'},
   'Apple iPhone 14 (128GB, Starlight)': {'5 Stars': '87',
    '4 Stars': '9',
    '3 Stars': '1',
    '2 Stars': '2',
    '1 Stars': '2'},
   'Apple iPhone 14 (128GB, Purple)': {'5 Stars': '98',
    '4 Stars': '7',
    '3 Stars': '2',
    '2 Stars': '1',
    '1 Stars': 0},
   'Apple iPhone 14 (128GB, Midnight)': {'5 Stars': '106',
    '4 Stars': '18',
    '3 Stars': '2',
    '2 Stars': '3',
    '1 Stars': 0},
   'Apple iPhone 14 (256GB, Blue)': {'5 Stars': '28',
    '4 Stars': '1',
    '3 Stars': '1',
    '2 Stars': 0,
    '1 Stars': 0},
   'Apple iPhone 14 (128GB, Red)': {'5 Stars': '24',
    '4 Stars': '2',
    '3 Stars': '1',
    '2 Stars': 0,
    '1 Stars': 0},
   'Apple iPhone 14 (256GB, Red)': {'5 Stars': '2',
    '4 Stars': '2',
    '3 Stars': 0,
    '2 Stars': 0,
    '1 Stars': 0},
   'Apple iPhone 14 (256GB, Purple)': {'5 Stars': '12',
    '

In [32]:

## cleaning class

import re
import pandas as pd
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


count = 0
length = 0

class Cleaning:
    def __init__(self, review_of_product, price_product, overall_r, ratings_of_product, link_product):
        self.review_of_product = review_of_product
        self.price_product = price_product
        self.overall_r = overall_r
        self.ratings_of_product = ratings_of_product
        self.link_product = link_product
        
        self.df = pd.DataFrame(
            {'Product Name': [], 'Review': [], 'Link': [], '5 Stars': [], '4 Stars': [], '3 Stars': [], '2 Stars': [],
             '1 Stars': [], 'Price': [], 'Overall Rating': []})

    def clean(self, review_of_product, ratings_of_product, link_product, price_product, overall_r):
        count = 0
        length = 0
        length1 = 0
        for i in self.review_of_product:
            for j in i:
                # length = 0# y is scraped data
                for k, l in j.items():
                    for m in k:
                        self.df.loc[count] = [k, m, self.link_product[length1][length][k], self.ratings_of_product[length1][length][k]['5 Stars'],
                                         self.ratings_of_product[length1][length][k]['4 Stars'], self.ratings_of_product[length1][length][k]['3 Stars'],
                                         self.ratings_of_product[length1][length][k]['2 Stars'], self.ratings_of_product[length1][length][k]['1 Stars'],
                                         self.price_product[length1][length][k], self.overall_r[length1][length][k]]
                        count += 1
                length += 1
            length1 += 1
        
        
        # cleaning rating 
        x = self.df['Avg Rating']
        index = 0
        for i in x:
            if type(i) == list:
                y = i[0].strip()
                print(float(y))
                self.df.loc[index, 'Avg Rating'] = y
            else:
                if type(i) == str:
                    string = ' '
                    for j in i:
                        if  j.isalpha():
                            continue
                        string+= j
                    string = string.strip()
                    y = float(string[:4])
                    print(y)
                    self.df.loc[index, 'Avg Rating'] = y
            index+=1
            
        
        # cleeaning price
        x = self.df['Price']
        index  = 0
        for i in x:
            if type(i) == list:
                y = i[0]
                y = y.split('.')
                y =y[0]
                y = re.findall(r'\d', y)
                string = ' '
                for j in y:
                    string+=j
                y = float(string)
                print(y)
                self.df.loc[index, 'Price'] = y
            else:
                y = i
                if '\n' in y:
                    y = y.split('\n')
                    y = y[0]
                y = y.split('.')
                y = y[0]
                y = re.sub(r'[\n₹,]', '', y)
                string = ' '
                for j in y:
                    string+=j
                if len(string) == 0 or len(string) == 1:
                    string = 0
                else:
                    y = float(string)
                print(y)
                self.df.loc[index, 'Price'] = y

            index += 1
            
         # cleaning all stars
        for i in range(len(df)):
            count = 5
            total = 0
            while count != 0:
                x = df['{} Stars'.format(count)][i]
                x = str(x)
                x = x.strip()
                x = re.sub(r'[,\n%]', '', x)
                x = int(x)
                total+=x
                df.loc[i, '{} Stars'.format(count)] = x
                count-=1
            count1 = 5
            while count1!= 0:
                x = df['{} Stars'.format(count1)][i]
                percent = (x / total) * 100
                df.loc[i, '{} Stars'.format(count1)] = percent
                count1 -= 1
        
        
        # clean review
        cleaned = []
        for i in self.df['Review']:
            x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
            y = re.sub(r'READ MORE', '', x)
            # y = re.sub(r'[^\x00-\x7F]+', '', x)
            cleaned.append(y)

        token = []
        stop_words = set(stopwords.words('english'))
        for i in cleaned:
            word_tokens = word_tokenize(i)
            filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
            word = ' '.join(filtered_words)
            words = re.sub('[^a-zA-Z0-9 \n]', '', word)

            print(words)
            token.append(words)
        self.df['Cleaned'] = token

        return self.df

In [33]:
df = Cleaning(review, price, avg_rating, all_rating, link)

In [73]:
import pandas as pd
import re

In [97]:
df = pd.DataFrame(
            {'Product Name': [], 'Review': [], 'Link': [], '5 Stars': [], '4 Stars': [], '3 Stars': [], '2 Stars': [],
             '1 Stars': [], 'Avg Rating': [], 'Price': [], 'Overall Rating': []})

def clean( review_of_product, ratings_of_product, link_product, price_product, overall_r):
    df = pd.DataFrame({'Product' : [], 'Review':[], "5 Stars":[], '4 Stars' : [], '3 Stars' : [], '2 Stars' : [], '1 Stars' : [], 'Avg Rating' : [], "Price" : [], 'Link' : [] } )
    count = 0
    length1 = 0

    for i in review:
        length = 0
        for j in i:
            #length = 0
            for k, l in j.items():
                #print(k)
                for m in l:
                    df.loc[count] = [k, m, all_rating[length1][length][k]['5 Stars'], all_rating[length1][length][k]['4 Stars'], all_rating[length1][length][k]['3 Stars'], all_rating[length1][length][k]['2 Stars'], all_rating[length1][length][k]['1 Stars'], avg_rating[length1][length][k], price[length1][length][k], link[length1][length][k] ]
                    count += 1
            length += 1
        length1 +=1
    
    

    cleaned = []
    for i in df['Review']:
        x = re.sub(r"[^a-zA-Z\s.,!?']", "", i)
        y = re.sub(r'READ MORE', '', x)
        # y = re.sub(r'[^\x00-\x7F]+', '', x)
        cleaned.append(y)

    token = []
    stop_words = set(stopwords.words('english'))
    for i in cleaned:
        word_tokens = word_tokenize(i)
        filtered_words = [word for word in word_tokens if not word.lower() in stop_words]
        word = ' '.join(filtered_words)
        words = re.sub('[^a-zA-Z0-9 \n]', '', word)

        print(words)
        token.append(words)
    df['Cleaned'] = token
    return df

In [98]:
df = clean(review, price, avg_rating, all_rating, link)

NameError: name 'review' is not defined

In [85]:
df.to_csv('All Review of Iphone 14', index = False)

In [84]:
df

,Product,Review,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Avg Rating,Price,Link,Cleaned
0,"Apple iPhone 14 (128GB, Blue)",Its a very good product.,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...,good product
1,"Apple iPhone 14 (128GB, Blue)",So far good,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...,far good
2,"Apple iPhone 14 (128GB, Blue)",Wonderful,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...,Wonderful
3,"Apple iPhone 14 (128GB, Blue)",Good,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...,Good
4,"Apple iPhone 14 (128GB, Blue)",Very good performance,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...,good performance
...,...,...,...,...,...,...,...,...,...,...,...
2992,Apple iPhone 14 Pro Max (1 TB) - Space Black,,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...,
2993,Apple iPhone 14 Pro Max (1 TB) - Space Black,This is my Personal & primary (main) MobileNew...,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...,Personal primary main MobileNew Update Pro Max...
2994,Apple iPhone 14 Pro Max (1 TB) - Space Black,Insane battery life . I'm happy with purchase....,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...,Insane battery life m happy purchase Volume ...
2995,Apple iPhone 14 Pro Max (1 TB) - Space Black,"Specifications good,but seems repeated you may...",49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...,Specifications good seems repeated may go wai...


In [186]:
df = pd.DataFrame({'Product' : [], 'Review':[], "5 Stars":[], '4 Stars' : [], '3 Stars' : [], '2 Stars' : [], '1 Stars' : [], 'Avg Rating' : [], "Price" : [], 'Link' : [] } )
count = 0
length1 = 0

for i in review:
    length = 0
    for j in i:
        #length = 0
        for k, l in j.items():
            #print(k)
            for m in l:
                df.loc[count] = [k, m, all_rating[length1][length][k]['5 Stars'], all_rating[length1][length][k]['4 Stars'], all_rating[length1][length][k]['3 Stars'], all_rating[length1][length][k]['2 Stars'], all_rating[length1][length][k]['1 Stars'], avg_rating[length1][length][k], price[length1][length][k], link[length1][length][k] ]
                count += 1
        length += 1
    length1 +=1
df    
            

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\dtypes\cast.py:883: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


,Product,Review,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Avg Rating,Price,Link
0,"Apple iPhone 14 (128GB, Blue)",Its a very good product.,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...
1,"Apple iPhone 14 (128GB, Blue)",So far good,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...
2,"Apple iPhone 14 (128GB, Blue)",Wonderful,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...
3,"Apple iPhone 14 (128GB, Blue)",Good,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...
4,"Apple iPhone 14 (128GB, Blue)",Very good performance,172,20,2,4,6,[4.7 ],"[ ₹70,990.00 ]",https://www.croma.com/apple-iphone-14-128gb-bl...
...,...,...,...,...,...,...,...,...,...,...
2992,Apple iPhone 14 Pro Max (1 TB) - Space Black,,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2993,Apple iPhone 14 Pro Max (1 TB) - Space Black,This is my Personal & primary (main) MobileNew...,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2994,Apple iPhone 14 Pro Max (1 TB) - Space Black,Insane battery life . I'm happy with purchase....,49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2995,Apple iPhone 14 Pro Max (1 TB) - Space Black,"Specifications good,but seems repeated you may...",49%,8%,21%,0%,22%,3.6 out of 5,"₹1,89,900",https://www.amazon.in/Apple-iPhone-Pro-Space-B...


In [44]:
for i in review:
    for j in i:
        print(len(j))

20
20
15
24
24
24
17
5
5


In [195]:
# cleaning avg rating
x = df['Avg Rating']
index = 0
for i in x:
    if type(i) == list:
        y = i[0].strip()
        print(float(y))
        df.loc[index, 'Avg Rating'] = y
    else:
        if type(i) == str:
            string = ' '
            for j in i:
                if  j.isalpha():
                    continue
                string+= j
            string = string.strip()
            y = float(string[:4])
            print(y)
            df.loc[index, 'Avg Rating'] = y
    index+=1


4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.5
4.5
4.5
4.5
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.3
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.9
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8
4.8


4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.7
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6
4.6


In [198]:
# cleaning price
x = df['Price']
index  = 0
for i in x:
    if type(i) == list:
        y = i[0]
        y = y.split('.')
        y =y[0]
        y = re.findall(r'\d', y)
        string = ' '
        for j in y:
            string+=j
        y = float(string)
        print(y)
        df.loc[index, 'Price'] = y
    else:
        if type(y) !=  float:
            
            #if type(y) != float:
            if '\n' in y:
                y = y.split('\n')
                y = y[0]
            y = y.split('.')
            y = y[0]
            y = re.sub(r'[\n₹,]', '', y)
            string = ' '
            for j in y:
                string+=j
            if len(string) == 0 or len(string) == 1:
                string = 0
            else:
                y = float(string)
            print(y)
        df.loc[index, 'Price'] = y
        
    
    
    index += 1
df    

AttributeError: 'float' object has no attribute 'split'

['₹86,999', '₹99,900']

In [192]:
count = 0
for i in df['Price']:
    print(count, i)
    count+=1

0 70990.0
1 70990.0
2 70990.0
3 70990.0
4 70990.0
5 70990.0
6 70990.0
7 70990.0
8 70990.0
9 70990.0
10 70990.0
11 70990.0
12 70990.0
13 70990.0
14 70990.0
15 70990.0
16 70990.0
17 70990.0
18 70990.0
19 70990.0
20 70990.0
21 70990.0
22 70990.0
23 70990.0
24 70990.0
25 70990.0
26 70990.0
27 70990.0
28 70990.0
29 70990.0
30 70990.0
31 70990.0
32 70990.0
33 70990.0
34 70990.0
35 70990.0
36 70990.0
37 70990.0
38 70990.0
39 70990.0
40 70990.0
41 70990.0
42 70990.0
43 70990.0
44 70990.0
45 70990.0
46 70990.0
47 70990.0
48 70990.0
49 70990.0
50 70990.0
51 70990.0
52 70990.0
53 70990.0
54 70990.0
55 70990.0
56 70990.0
57 70990.0
58 70990.0
59 70990.0
60 70990.0
61 70990.0
62 70990.0
63 70990.0
64 70990.0
65 70990.0
66 70990.0
67 70990.0
68 70990.0
69 70990.0
70 70990.0
71 70990.0
72 70990.0
73 70990.0
74 70990.0
75 70990.0
76 70990.0
77 70990.0
78 70990.0
79 70990.0
80 70990.0
81 70990.0
82 70990.0
83 70990.0
84 70990.0
85 70990.0
86 70990.0
87 70990.0
88 70990.0
89 70990.0
90 70990.0
91 70990.

1124 99999.0
1125 99999.0
1126 99999.0
1127 99999.0
1128 99999.0
1129 99999.0
1130 99999.0
1131 99999.0
1132 99999.0
1133 99999.0
1134 99999.0
1135 99999.0
1136 99999.0
1137 99999.0
1138 99999.0
1139 99999.0
1140 99999.0
1141 99999.0
1142 99999.0
1143 99999.0
1144 99999.0
1145 99999.0
1146 99999.0
1147 99999.0
1148 99999.0
1149 99999.0
1150 99999.0
1151 99999.0
1152 99999.0
1153 99999.0
1154 99999.0
1155 99999.0
1156 99999.0
1157 99999.0
1158 99999.0
1159 99999.0
1160 99999.0
1161 99999.0
1162 99999.0
1163 99999.0
1164 99999.0
1165 99999.0
1166 99999.0
1167 99999.0
1168 99999.0
1169 99999.0
1170 99999.0
1171 99999.0
1172 99999.0
1173 99999.0
1174 79999.0
1175 79999.0
1176 79999.0
1177 79999.0
1178 79999.0
1179 79999.0
1180 79999.0
1181 79999.0
1182 79999.0
1183 79999.0
1184 79999.0
1185 79999.0
1186 79999.0
1187 79999.0
1188 79999.0
1189 79999.0
1190 79999.0
1191 79999.0
1192 79999.0
1193 79999.0
1194 79999.0
1195 79999.0
1196 79999.0
1197 79999.0
1198 79999.0
1199 79999.0
1200 79999.0

2378 42999.0
2379 42999.0
2380 42999.0
2381 42999.0
2382 42999.0
2383 42999.0
2384 42999.0
2385 42999.0
2386 42999.0
2387 42999.0
2388 42999.0
2389 42999.0
2390 42999.0
2391 42999.0
2392 42999.0
2393 42999.0
2394 42999.0
2395 42999.0
2396 42999.0
2397 42999.0
2398 42999.0
2399 42999.0
2400 42999.0
2401 42999.0
2402 42999.0
2403 42999.0
2404 38999.0
2405 38999.0
2406 38999.0
2407 38999.0
2408 38999.0
2409 38999.0
2410 38999.0
2411 38999.0
2412 38999.0
2413 38999.0
2414 38999.0
2415 38999.0
2416 38999.0
2417 38999.0
2418 38999.0
2419 38999.0
2420 38999.0
2421 38999.0
2422 38999.0
2423 38999.0
2424 38999.0
2425 38999.0
2426 38999.0
2427 38999.0
2428 38999.0
2429 38999.0
2430 38999.0
2431 38999.0
2432 38999.0
2433 38999.0
2434 38999.0
2435 38999.0
2436 38999.0
2437 38999.0
2438 38999.0
2439 38999.0
2440 38999.0
2441 38999.0
2442 38999.0
2443 38999.0
2444 38999.0
2445 38999.0
2446 38999.0
2447 38999.0
2448 38999.0
2449 38999.0
2450 38999.0
2451 38999.0
2452 38999.0
2453 38999.0
2454 38999.0

In [196]:
df

,Product,Review,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Avg Rating,Price,Link
0,"Apple iPhone 14 (128GB, Blue)",Its a very good product.,172,20,2,4,6,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
1,"Apple iPhone 14 (128GB, Blue)",So far good,172,20,2,4,6,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
2,"Apple iPhone 14 (128GB, Blue)",Wonderful,172,20,2,4,6,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
3,"Apple iPhone 14 (128GB, Blue)",Good,172,20,2,4,6,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
4,"Apple iPhone 14 (128GB, Blue)",Very good performance,172,20,2,4,6,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
...,...,...,...,...,...,...,...,...,...,...
2992,Apple iPhone 14 Pro Max (1 TB) - Space Black,,49%,8%,21%,0%,22%,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2993,Apple iPhone 14 Pro Max (1 TB) - Space Black,This is my Personal & primary (main) MobileNew...,49%,8%,21%,0%,22%,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2994,Apple iPhone 14 Pro Max (1 TB) - Space Black,Insane battery life . I'm happy with purchase....,49%,8%,21%,0%,22%,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2995,Apple iPhone 14 Pro Max (1 TB) - Space Black,"Specifications good,but seems repeated you may...",49%,8%,21%,0%,22%,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...


In [123]:
import pandas as pd
df= pd.read_csv('All Review of Iphone 14 cleaned')


In [3]:
prod = df.Product.unique()

In [14]:
import numpy as np
x = df.loc[0]
a, b,c,d,e = int(x['5 Stars']),int(x['4 Stars']),int(x['3 Stars']),int(x['2 Stars']),int(x['1 Stars']) 

percent = a/(a+b+c+d+e)


84.31372549019608

In [125]:
# cleaning stars
import re
for i in range(len(df)):
    count = 5
    total = 0
    while count != 0:
        x = df['{} Stars'.format(count)][i]
        x = str(x)
        x = x.strip()
        x = re.sub(r'[,\n%]', '', x)
        x = int(x)
        total+=x
        df.loc[i, '{} Stars'.format(count)] = x
        count-=1
    count1 = 5
    while count1!= 0:
        x = df['{} Stars'.format(count1)][i]
        percent = (x / total) * 100
        df.loc[i, '{} Stars'.format(count1)] = percent
        count1 -= 1
        
    
df   
    

,Product,Review,5 Stars,4 Stars,3 Stars,2 Stars,1 Stars,Avg Rating,Price,Link
0,"Apple iPhone 14 (128GB, Blue)",Its a very good product.,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
1,"Apple iPhone 14 (128GB, Blue)",So far good,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
2,"Apple iPhone 14 (128GB, Blue)",Wonderful,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
3,"Apple iPhone 14 (128GB, Blue)",Good,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
4,"Apple iPhone 14 (128GB, Blue)",Very good performance,84.313725,9.803922,0.980392,1.960784,2.941176,4.7,70990.0,https://www.croma.com/apple-iphone-14-128gb-bl...
...,...,...,...,...,...,...,...,...,...,...
2992,Apple iPhone 14 Pro Max (1 TB) - Space Black,NaN,49.0,8.0,21.0,0.0,22.0,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2993,Apple iPhone 14 Pro Max (1 TB) - Space Black,This is my Personal & primary (main) MobileNew...,49.0,8.0,21.0,0.0,22.0,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2994,Apple iPhone 14 Pro Max (1 TB) - Space Black,Insane battery life . I'm happy with purchase....,49.0,8.0,21.0,0.0,22.0,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...
2995,Apple iPhone 14 Pro Max (1 TB) - Space Black,"Specifications good,but seems repeated you may...",49.0,8.0,21.0,0.0,22.0,3.6,189900.0,https://www.amazon.in/Apple-iPhone-Pro-Space-B...


20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
9
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
7
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
18
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
7
7
7
7
7
7
7
7
7
7
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
10
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


112
112
112
112
112
112
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
112
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
98
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,563
24,56

In [51]:
import plotly.graph_objs as go

data = [30, 20, 50, 10, 10]  # Example data
labels = ['A', 'B', 'C', 'D', 'E']  # Example labels

fig = go.Figure(data=[go.Pie(labels=labels, values=data)])
#fig.update_traces(hovertemplate='%{y:.2f}%')

fig.show()

In [91]:
x = df.Link[2990]
x  = re.sub(r'https://www.', '', x)
try:
    x = x[:x.index('.com')] 
except ValueError:
    x = x[:x.index('.in')] 
x

'amazon'

In [93]:
df['Website'] = pd.Series([])

C:\Users\HASHIM RAZI\AppData\Local\Temp\ipykernel_20500\2793719318.py:1: FutureWarning:

The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.



In [102]:
for i, j in enumerate(df.Link):
    x  = re.sub(r'https://www.', '', j)
    try:
        x = x[:x.index('.com')] 
    except ValueError:
        x = x[:x.index('.in')] 
    df.loc[i, "Website"] = x

In [107]:
x = df[df['Website'] == 'croma']


4.652777777777751

In [111]:
x = df[df['Website'] == 'flipkart']
x['Price'].mean()

96471.22222222222

In [119]:
# finding index
y = x['Price'].max()
df.index[df.Price == y]

Int64Index([2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2204,
            2205, 2206, 2207, 2208, 2209, 2210, 2211, 2212, 2213, 2214, 2215,
            2216, 2217, 2218, 2219, 2220, 2221, 2222, 2223],
           dtype='int64')